In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
video_id="dDP36_ZBs6A"
t=YouTubeTranscriptApi.get_transcript(video_id)

In [ ]:
transcript = " ".join([segment['text'] for segment in t])
print(transcript)

In [ ]:
import requests
import time
import json

def split_text_into_chunks(text, max_words=30):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunk = ' '.join(words[i:i + max_words])
        chunks.append(chunk.strip())
    return chunks

def generate_questions_and_options(chunk):
    url = "https://api.llama.ai/v1/completions"
    headers = {
        "Authorization": "LA-b24266d1675c4f5dbd70417e330e52ebc104a661536e476a81bc48b4c5507230",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "llama-13b",
        "prompt": f"Generate a question from the following text chunk:\n\n{chunk}\n\nProvide 4 options, with only 1 correct option.",
        "max_tokens": 150,
        "temperature": 0.7
    }
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        response_data = response.json()
        print("API Response:", response_data)
        choices = response_data.get('choices')
        if choices:
            text = choices[0].get('text', '')
            return text
        else:
            return "No choices available in the response."
    
    except requests.exceptions.RequestException as e:
        print(f"Error during API request: {e}")
        return "Error generating question and options."

def store_data(question, options, answer, index):
    data = {
        'question': question,
        'options': options,
        'answer': answer,
        'index': index
    }
    return data

# Main logic
text = transcript
chunks = split_text_into_chunks(text, max_words=30)
questions_data = []

for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:\n{chunk}\n")
    question_and_options = generate_questions_and_options(chunk)
    question = "Parsed question from API response"
    options = ["Option 1", "Option 2", "Option 3", "Option 4"]
    answer = "Correct option"
    
    question_data = store_data(question, options, answer, i)
    questions_data.append(question_data)

# Save questions data to a JSON file
output_filename = "questions_data.json"
with open(output_filename, 'w') as json_file:
    json.dump(questions_data, json_file, indent=4)

print(f"Collected data has been saved to {output_filename}")


In [ ]:
from dotenv import load_dotenv
import google.generativeai as genai
from google.generativeai import GenerationConfig
import json
import os
import time

# Load environment variables, particularly GEMINI_KEY
load_dotenv()

# Configure Gemini API
genai.configure(api_key=os.getenv("GEMINI_KEY"))
config = GenerationConfig(temperature=0.9)




In [ ]:
def split_text_into_chunks(text, max_words=30):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunk = ' '.join(words[i:i + max_words])
        chunks.append(chunk.strip())
    return chunks

def generate_questions_and_options(chunk):
    try:
        prompt = f"Generate a question from the following text chunk:\n\n{chunk}\n\nProvide 4 options, with only 1 correct option."
        
        # Call the Gemini API
        response = genai.GenerativeModel("gemini-1.5-flash",
                                         system_instruction=prompt,
                                         generation_config=config)
        result = response.generate_content(prompt)
        
        # Parse the response to extract question and options
        # print("API Response:", result)
        question_and_options_text = result.text
        print(question_and_options_text)  # Assuming this is where the response text is stored

        # Here you should parse `question_and_options_text` to extract the actual question, options, and answer.
        # For demonstration, assume the question and options are extracted as below:
        question = "Parsed question from API response"  # Replace with actual parsed question
        options = ["Option 1", "Option 2", "Option 3", "Option 4"]  # Replace with parsed options
        answer = "Correct option"  # Replace with the actual correct answer

        return question, options, answer

    except Exception as e:
        print(f"Error during API request: {e}")
        return "Error generating question and options.", [], ""

def store_data(question, options, answer, index):
    data = {
        'question': question,
        'options': options,
        'answer': answer,
        'index': index
    }
    return data


In [ ]:

# Sample text transcript
text = transcript
chunks = split_text_into_chunks(text, max_words=30)
questions_data = []

# Process each chunk
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:\n{chunk}\n")
    question, options, answer = generate_questions_and_options(chunk)
    # question_data = store_data(question, options, answer, i)
    # questions_data.append(question_data)
    print(question, options, answer)
    break
# # Save to JSON file
# with open("questions_data.json", "w") as json_file:
#     json.dump(questions_data, json_file, indent=4)

# print("Collected data has been saved to questions_data.json")